In [1]:
from pyspark.sql import SparkSession

spark = SparkSession.builder  \
        .master('local[*]') \
        .config('spark.driver.memory', '32g') \
        .config("spark.jars.packages", "com.microsoft.azure:synapseml_2.12:0.11.1") \
        .getOrCreate()
spark

:: loading settings :: url = jar:file:/workspace/apache-spark-ml/.venv/lib/python3.8/site-packages/pyspark/jars/ivy-2.5.1.jar!/org/apache/ivy/core/settings/ivysettings.xml


Ivy Default Cache set to: /root/.ivy2/cache
The jars for the packages stored in: /root/.ivy2/jars
com.microsoft.azure#synapseml_2.12 added as a dependency
:: resolving dependencies :: org.apache.spark#spark-submit-parent-6b0db135-d1dd-411c-bd91-6c0ed5bab21b;1.0
	confs: [default]
	found com.microsoft.azure#synapseml_2.12;0.11.1 in central
	found com.microsoft.azure#synapseml-core_2.12;0.11.1 in central
	found org.scalactic#scalactic_2.12;3.2.14 in central
	found org.scala-lang#scala-reflect;2.12.15 in central
	found io.spray#spray-json_2.12;1.3.5 in central
	found com.jcraft#jsch;0.1.54 in central
	found org.apache.httpcomponents.client5#httpclient5;5.1.3 in central
	found org.apache.httpcomponents.core5#httpcore5;5.1.3 in central
	found org.apache.httpcomponents.core5#httpcore5-h2;5.1.3 in central
	found org.slf4j#slf4j-api;1.7.25 in central
	found commons-codec#commons-codec;1.15 in central
	found org.apache.httpcomponents#httpmime;4.5.13 in central
	found org.apache.httpcomponents#ht

In [2]:
santander_sdf = spark.read.csv('data/train_santander.csv', header=True, inferSchema=True)
santander_sdf.show()

23/06/08 06:14:53 WARN package: Truncated the string representation of a plan since it was too large. This behavior can be adjusted by setting 'spark.sql.debug.maxToStringFields'.


+---+----+-----+------------------+-----------------------+-----------------------+-----------------------+-----------------------+-----------------------+-----------------------+-----------------+-----------------------+-----------------------+-----------------------+-----------------------+-----------------+-----------------------+-----------------------+-----------------+------------------+----------+--------+----------+--------+----------+--------+----------+--------+----------+--------+-----------+---------+-----------+-----------------+---------------+-----------------+---------------+-----------------+---------------+---------+-----------+---------+-----------+---------+-----------+---------+---------+-----------+---------+-----------+---------+-------------+-----------+-------------+---------+-----------+---------+-----------+-----------+---------+---------+-----------+---------+-----------+---------+-----------+---------+-------------+-----------+---------+-----------+--------

In [3]:
santander_sdf.count()

76020

In [5]:
import pyspark.sql.functions as F

santander_sdf.printSchema()
santander_sdf.select([F.count(F.when(F.col(c).isNull() | F.isnan(c), c)).alias(c) for c in santander_sdf.columns]).show()


root
 |-- ID: integer (nullable = true)
 |-- var3: integer (nullable = true)
 |-- var15: integer (nullable = true)
 |-- imp_ent_var16_ult1: double (nullable = true)
 |-- imp_op_var39_comer_ult1: double (nullable = true)
 |-- imp_op_var39_comer_ult3: double (nullable = true)
 |-- imp_op_var40_comer_ult1: double (nullable = true)
 |-- imp_op_var40_comer_ult3: double (nullable = true)
 |-- imp_op_var40_efect_ult1: double (nullable = true)
 |-- imp_op_var40_efect_ult3: double (nullable = true)
 |-- imp_op_var40_ult1: double (nullable = true)
 |-- imp_op_var41_comer_ult1: double (nullable = true)
 |-- imp_op_var41_comer_ult3: double (nullable = true)
 |-- imp_op_var41_efect_ult1: double (nullable = true)
 |-- imp_op_var41_efect_ult3: double (nullable = true)
 |-- imp_op_var41_ult1: double (nullable = true)
 |-- imp_op_var39_efect_ult1: double (nullable = true)
 |-- imp_op_var39_efect_ult3: double (nullable = true)
 |-- imp_op_var39_ult1: double (nullable = true)
 |-- imp_sal_var16_ult1: dou

+---+----+-----+------------------+-----------------------+-----------------------+-----------------------+-----------------------+-----------------------+-----------------------+-----------------+-----------------------+-----------------------+-----------------------+-----------------------+-----------------+-----------------------+-----------------------+-----------------+------------------+----------+--------+----------+--------+----------+--------+----------+--------+----------+--------+-----------+---------+-----------+-----------------+---------------+-----------------+---------------+-----------------+---------------+---------+-----------+---------+-----------+---------+-----------+---------+---------+-----------+---------+-----------+---------+-------------+-----------+-------------+---------+-----------+---------+-----------+-----------+---------+---------+-----------+---------+-----------+---------+-----------+---------+-------------+-----------+---------+-----------+--------

In [5]:
len(santander_sdf.columns), santander_sdf.count()

(371, 76020)

In [6]:
santander_sdf.describe().show()

23/06/08 05:12:54 WARN DAGScheduler: Broadcasting large task binary with size 1346.0 KiB
23/06/08 05:13:20 WARN DAGScheduler: Broadcasting large task binary with size 3.6 MiB


+-------+-----------------+-------------------+------------------+------------------+-----------------------+-----------------------+-----------------------+-----------------------+-----------------------+-----------------------+-----------------+-----------------------+-----------------------+-----------------------+-----------------------+------------------+-----------------------+-----------------------+-----------------+------------------+--------------------+--------------------+----------+--------+------------------+------------------+--------------------+--------------------+--------------------+--------------------+-------------------+--------------------+-------------------+-------------------+--------------------+--------------------+--------------------+--------------------+--------------------+-------------------+-------------------+-------------------+--------------------+--------------------+--------------------+--------------------+--------------------+------------------

In [7]:
santander_sdf.groupBy('var3').count().orderBy(F.col('count').desc()).show()

+-------+-----+
|   var3|count|
+-------+-----+
|      2|74165|
|      8|  138|
|-999999|  116|
|      9|  110|
|      3|  108|
|      1|  105|
|     13|   98|
|      7|   97|
|      4|   86|
|     12|   85|
|      6|   82|
|      0|   75|
|     10|   72|
|     11|   66|
|      5|   63|
|     14|   61|
|     15|   34|
|     18|   10|
|     16|    9|
|     17|    7|
+-------+-----+
only showing top 20 rows



In [8]:
santander_sdf = santander_sdf.withColumn('var3', F.when(F.col('var3') == -999999, 2).otherwise(F.col('var3')))
santander_sdf.groupBy('var3').count().orderBy(F.col('count').desc()).show()

+----+-----+
|var3|count|
+----+-----+
|   2|74281|
|   8|  138|
|   9|  110|
|   3|  108|
|   1|  105|
|  13|   98|
|   7|   97|
|   4|   86|
|  12|   85|
|   6|   82|
|   0|   75|
|  10|   72|
|  11|   66|
|   5|   63|
|  14|   61|
|  15|   34|
|  18|   10|
|  16|    9|
|  17|    7|
|  23|    7|
+----+-----+
only showing top 20 rows



In [9]:
santander_sdf = santander_sdf.drop('ID')
santander_sdf.describe().show()

23/06/08 05:13:33 WARN DAGScheduler: Broadcasting large task binary with size 1345.6 KiB
23/06/08 05:14:02 WARN DAGScheduler: Broadcasting large task binary with size 3.6 MiB


+-------+------------------+------------------+------------------+-----------------------+-----------------------+-----------------------+-----------------------+-----------------------+-----------------------+-----------------+-----------------------+-----------------------+-----------------------+-----------------------+------------------+-----------------------+-----------------------+-----------------+------------------+--------------------+--------------------+----------+--------+------------------+------------------+--------------------+--------------------+--------------------+--------------------+-------------------+--------------------+-------------------+-------------------+--------------------+--------------------+--------------------+--------------------+--------------------+-------------------+-------------------+-------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+----------------

In [10]:
santander_sdf.groupBy('TARGET').count().show()

+------+-----+
|TARGET|count|
+------+-----+
|     1| 3008|
|     0|73012|
+------+-----+



In [11]:
train_sdf, test_sdf = santander_sdf.randomSplit([0.8, 0.2], seed=2021)

In [12]:
train_cnt = train_sdf.count()
test_cnt = test_sdf.count()
print(type(train_cnt), type(test_cnt))
print(type(train_sdf.groupBy('TARGET').count()))
print(type(test_sdf.groupBy('TARGET').count()))

<class 'int'> <class 'int'>
<class 'pyspark.sql.dataframe.DataFrame'>
<class 'pyspark.sql.dataframe.DataFrame'>


In [13]:
train_target_count = train_sdf.groupBy('TARGET').count().alias('count')
train_target_count.withColumn('percent', F.col('count') / train_cnt).show()
test_target_count = test_sdf.groupBy('TARGET').count().alias('count')
test_target_count.withColumn('percent', F.col('count') / test_cnt).show()

+------+-----+--------------------+
|TARGET|count|             percent|
+------+-----+--------------------+
|     1| 2462|0.040364626028789714|
|     0|58532|  0.9596353739712102|
+------+-----+--------------------+



+------+-----+--------------------+
|TARGET|count|             percent|
+------+-----+--------------------+
|     1|  546|0.036337015839212033|
|     0|14480|   0.963662984160788|
+------+-----+--------------------+



In [14]:
zeros_sdf = santander_sdf.filter(F.col('TARGET') == 0)
ones_sdf = santander_sdf.filter(F.col('TARGET') == 1)
train_zeros_sdf, test_zeros_sdf = zeros_sdf.randomSplit([0.8, 0.2], seed=2001)
train_ones_sdf, test_ones_sdf = ones_sdf.randomSplit([0.8, 0.2], seed=2001)
train_sdf = train_zeros_sdf.union(train_ones_sdf)
test_sdf = test_zeros_sdf.union(test_ones_sdf)
train_cnt = train_sdf.count()
test_cnt = test_sdf.count()
train_target_count = train_sdf.groupBy('TARGET').count().alias('count')
train_target_count.withColumn('percent', F.col('count') / train_cnt).show()
test_target_count = test_sdf.groupBy('TARGET').count().alias('count')
test_target_count.withColumn('percent', F.col('count') / test_cnt).show()

+------+-----+--------------------+
|TARGET|count|             percent|
+------+-----+--------------------+
|     0|58330|  0.9602120269313711|
|     1| 2417|0.039787973068628905|
+------+-----+--------------------+



+------+-----+-------------------+
|TARGET|count|            percent|
+------+-----+-------------------+
|     0|14682| 0.9613042624238853|
|     1|  591|0.03869573757611471|
+------+-----+-------------------+



In [6]:
def stratified_train_test_split(data_sdf, label_name, split_ratio=[0.8, 0.2], seed=0):
    sdf_zeros = data_sdf.filter(F.col(label_name) == 0)
    sdf_ones = data_sdf.filter(F.col(label_name) == 1)
    sdf_zeros_train, sdf_zeros_test = sdf_zeros.randomSplit(split_ratio, seed=seed)
    sdf_ones_train, sdf_ones_test = sdf_ones.randomSplit(split_ratio, seed=seed)
    sdf_train = sdf_zeros_train.union(sdf_ones_train)
    sdf_test = sdf_zeros_test.union(sdf_ones_test)
    return sdf_train, sdf_test

In [7]:
train_sdf, test_sdf = stratified_train_test_split(santander_sdf, 'TARGET', seed=2021)

In [8]:
santander_sdf = spark.read.csv('data/train_santander.csv', header=True, inferSchema=True)
santander_sdf = santander_sdf.withColumn('var3', F.when(F.col('var3') == -999999, 2).otherwise(F.col('var3')))
santander_sdf = santander_sdf.drop('ID')
santander_sdf_columns = santander_sdf.columns
santander_sdf_columns.remove('TARGET')
sdf_train, sdf_test = stratified_train_test_split(santander_sdf, 'TARGET', seed=2021)

In [9]:
from pyspark.ml.feature import VectorAssembler

vector_assembler = VectorAssembler(inputCols=santander_sdf_columns, outputCol='features')
train_sdf_vectorized = vector_assembler.transform(sdf_train)
test_sdf_vectorized = vector_assembler.transform(sdf_test)
train_sdf_vectorized, val_sdf_vectorized = stratified_train_test_split(train_sdf_vectorized, 'TARGET', seed=2021)

In [22]:
train_sdf_vectorized.show()

+----+-----+------------------+-----------------------+-----------------------+-----------------------+-----------------------+-----------------------+-----------------------+-----------------+-----------------------+-----------------------+-----------------------+-----------------------+-----------------+-----------------------+-----------------------+-----------------+------------------+----------+--------+----------+--------+----------+--------+----------+--------+----------+--------+-----------+---------+-----------+-----------------+---------------+-----------------+---------------+-----------------+---------------+---------+-----------+---------+-----------+---------+-----------+---------+---------+-----------+---------+-----------+---------+-------------+-----------+-------------+---------+-----------+---------+-----------+-----------+---------+---------+-----------+---------+-----------+---------+-----------+---------+-------------+-----------+---------+-----------+---------+--

In [29]:
from hyperopt import fmin, tpe, hp, STATUS_OK, Trials
from xgboost.spark import SparkXGBClassifier
from pyspark.ml.evaluation import BinaryClassificationEvaluator

search_space = {
    'max_depth': hp.quniform('max_depth', 5, 10, 1),
    'colsample_bytree': hp.uniform('colsample_bytree', 0.6, 0.95),
    'min_child_weight': hp.quniform('min_child_weight', 1, 6, 1),
    'learning_rate': hp.uniform('learning_rate', 0.05, 0.2)
}
def objective_func(space):
    xgb_classifier = SparkXGBClassifier(
        n_estimators=50, max_depth=(int(space['max_depth'])),
        colsample_bytree=space['colsample_bytree'],
        min_child_weight=int(space['min_child_weight']),
        learning_rate=space['learning_rate'],
        label_col='TARGET',
        missing=0.0
    )
    xgb_model = xgb_classifier.fit(train_sdf_vectorized)
    predictions = xgb_model.transform(val_sdf_vectorized)
    roc_eval = BinaryClassificationEvaluator(labelCol='TARGET')
    roc_auc = roc_eval.evaluate(predictions)
    print('space:', space, 'roc_auc:', roc_auc)
    return {'loss': -1*roc_auc, 'status': STATUS_OK}
algo = tpe.suggest
xgb_trials = Trials()
xgb_best = fmin(fn=objective_func, space=search_space, algo=algo, max_evals=14, trials=xgb_trials)
print(xgb_best)
print(xgb_trials.results)

  0%|          | 0/14 [00:00<?, ?trial/s, best loss=?]

23/06/08 05:58:05 WARN DAGScheduler: Broadcasting large task binary with size 1213.6 KiB
23/06/08 05:58:15 WARN DAGScheduler: Broadcasting large task binary with size 1174.5 KiB
[05:58:17] task 0 got new rank 0                                    (0 + 1) / 1]
23/06/08 05:58:36 WARN DAGScheduler: Broadcasting large task binary with size 1062.0 KiB


space:                                                
{'colsample_bytree': 0.6725492847483635, 'learning_rate': 0.0503156421667237, 'max_depth': 8.0, 'min_child_weight': 5.0}
roc_auc:                                              
0.8317104975740205                                    
  7%|▋         | 1/14 [00:58<12:39, 58.41s/trial, best loss: -0.8317104975740205]

23/06/08 05:59:04 WARN DAGScheduler: Broadcasting large task binary with size 1213.6 KiB
23/06/08 05:59:15 WARN DAGScheduler: Broadcasting large task binary with size 1174.5 KiB
[05:59:18] task 0 got new rank 0                                    (0 + 1) / 1]


space:                                                                           
{'colsample_bytree': 0.9061873590738385, 'learning_rate': 0.1525354272685432, 'max_depth': 6.0, 'min_child_weight': 2.0}
roc_auc:                                                                         
0.8356665231288756                                                               
 14%|█▍        | 2/14 [01:53<11:17, 56.43s/trial, best loss: -0.8356665231288756]

...

space:                                                                            
{'colsample_bytree': 0.7128830475028746, 'learning_rate': 0.1329629010036928, 'max_depth': 7.0, 'min_child_weight': 4.0}
roc_auc:                                                                          
0.8372517599778594                                                                
 86%|████████▌ | 12/14 [10:57<01:46, 53.18s/trial, best loss: -0.8372517599778594]

23/06/08 06:09:00 WARN DAGScheduler: Broadcasting large task binary with size 1213.6 KiB
23/06/08 06:09:10 WARN DAGScheduler: Broadcasting large task binary with size 1174.5 KiB
[06:09:12] task 0 got new rank 0                                    (0 + 1) / 1]
23/06/08 06:09:29 WARN DAGScheduler: Broadcasting large task binary with size 1067.4 KiB


space:                                                                            
{'colsample_bytree': 0.8688470643612558, 'learning_rate': 0.1532490083335072, 'max_depth': 8.0, 'min_child_weight': 2.0}
roc_auc:                                                                          
0.8312049249145884                                                                
 93%|█████████▎| 13/14 [11:49<00:52, 52.81s/trial, best loss: -0.8372517599778594]

23/06/08 06:09:53 WARN DAGScheduler: Broadcasting large task binary with size 1213.6 KiB
23/06/08 06:10:01 WARN DAGScheduler: Broadcasting large task binary with size 1174.5 KiB
[06:10:04] task 0 got new rank 0                                    (0 + 1) / 1]
23/06/08 06:10:24 WARN DAGScheduler: Broadcasting large task binary with size 1210.2 KiB


space:                                                                            
{'colsample_bytree': 0.9008975680828246, 'learning_rate': 0.15453061320565964, 'max_depth': 9.0, 'min_child_weight': 1.0}
roc_auc:                                                                          
0.8277004440845763                                                                
100%|██████████| 14/14 [12:45<00:00, 54.69s/trial, best loss: -0.8372517599778594]
{'colsample_bytree': 0.7128830475028746, 'learning_rate': 0.1329629010036928, 'max_depth': 7.0, 'min_child_weight': 4.0}
[{'loss': -0.8317104975740205, 'status': 'ok'}, {'loss': -0.8356665231288756, 'status': 'ok'}, {'loss': -0.8318711501742223, 'status': 'ok'}, {'loss': -0.834362696680914, 'status': 'ok'}, {'loss': -0.8317373326408692, 'status': 'ok'}, {'loss': -0.8359174310039121, 'status': 'ok'}, {'loss': -0.8316160381387127, 'status': 'ok'}, {'loss': -0.834414577810155, 'status': 'ok'}, {'loss': -0.8359143896963362, 'status': 'ok'}, {'l

In [31]:
xgb_classifier_best = SparkXGBClassifier(
    n_estimators=50,
    max_depth=int(xgb_best['max_depth']),
    colsample_bytree=xgb_best['colsample_bytree'],
    min_child_weight=int(xgb_best['min_child_weight']),
    learning_rate=xgb_best['learning_rate'],
    label_col='TARGET',
    missing=0.0
)
xgb_best_model = xgb_classifier_best.fit(train_sdf_vectorized)
xgb_predictions = xgb_best_model.transform(test_sdf_vectorized)
roc_eval = BinaryClassificationEvaluator(labelCol='TARGET')
roc_auc = roc_eval.evaluate(xgb_predictions)
print('roc_auc:', roc_auc)

23/06/08 06:12:42 WARN DAGScheduler: Broadcasting large task binary with size 1213.6 KiB
23/06/08 06:12:53 WARN DAGScheduler: Broadcasting large task binary with size 1174.5 KiB
[06:12:58] task 0 got new rank 0                                    (0 + 1) / 1]


roc_auc: 0.8333743230092238


In [13]:
from hyperopt import fmin, tpe, hp, STATUS_OK, Trials
from pyspark.ml.evaluation import BinaryClassificationEvaluator
from synapse.ml.lightgbm import LightGBMClassifier

search_space = {
    'maxDepth': hp.quniform('maxDepth', 10, 20, 1),
    'numLeaves': hp.quniform('numLeaves', 32, 78, 4),
    'baggingFraction': hp.uniform('baggingFraction', 0.6, 1.0),
    'featureFraction': hp.uniform('featureFraction', 0.6, 1.0),
    'learningRate': hp.uniform('learningRate', 0.01, 0.1)
}
def objective_func(space):
    lgbm_classifier = LightGBMClassifier(
        numIterations=50, maxDepth=(int(space['maxDepth'])),
        numLeaves=space['numLeaves'],
        baggingFraction=space['baggingFraction'],
        featureFraction=space['featureFraction'],
        learningRate=space['learningRate'],
        labelCol='TARGET'
    )
    lgbm_model = lgbm_classifier.fit(train_sdf_vectorized)
    predictions = lgbm_model.transform(val_sdf_vectorized)
    roc_eval = BinaryClassificationEvaluator(labelCol='TARGET')
    roc_auc = roc_eval.evaluate(predictions)
    print('space:', space, 'roc_auc:', roc_auc)
    return {'loss': -1*roc_auc, 'status': STATUS_OK}
algo = tpe.suggest
lgbm_trials = Trials()
lgbm_best = fmin(fn=objective_func, space=search_space, algo=algo, max_evals=14, trials=lgbm_trials)
print(lgbm_best)
print(lgbm_trials.results)

  0%|          | 0/14 [00:00<?, ?trial/s, best loss=?]

23/06/08 06:18:01 WARN BulkPartitionTask: LightGBM task encountered empty partition, for best performance ensure no partitions are empty
23/06/08 06:18:01 WARN BulkPartitionTask: LightGBM task encountered empty partition, for best performance ensure no partitions are empty
23/06/08 06:18:01 WARN BulkPartitionTask: LightGBM task encountered empty partition, for best performance ensure no partitions are empty
23/06/08 06:18:01 WARN BulkPartitionTask: LightGBM task encountered empty partition, for best performance ensure no partitions are empty
23/06/08 06:18:01 WARN BulkPartitionTask: LightGBM task encountered empty partition, for best performance ensure no partitions are empty
23/06/08 06:18:01 WARN BulkPartitionTask: LightGBM task encountered empty partition, for best performance ensure no partitions are empty
23/06/08 06:18:01 WARN BulkPartitionTask: LightGBM task encountered empty partition, for best performance ensure no partitions are empty
23/06/08 06:18:01 WARN BulkPartitionTask:

space:                                                
{'baggingFraction': 0.6256204681594901, 'featureFraction': 0.7012252912822381, 'learningRate': 0.011304976713787787, 'maxDepth': 13.0, 'numLeaves': 68.0}
roc_auc:                                              
0.8307837932655073                                    
  7%|▋         | 1/14 [01:29<19:24, 89.61s/trial, best loss: -0.8307837932655073]

23/06/08 06:19:21 WARN BulkPartitionTask: LightGBM task encountered empty partition, for best performance ensure no partitions are empty
23/06/08 06:19:21 WARN BulkPartitionTask: LightGBM task encountered empty partition, for best performance ensure no partitions are empty
23/06/08 06:19:21 WARN BulkPartitionTask: LightGBM task encountered empty partition, for best performance ensure no partitions are empty
23/06/08 06:19:21 WARN BulkPartitionTask: LightGBM task encountered empty partition, for best performance ensure no partitions are empty
23/06/08 06:19:21 WARN BulkPartitionTask: LightGBM task encountered empty partition, for best performance ensure no partitions are empty
23/06/08 06:19:21 WARN BulkPartitionTask: LightGBM task encountered empty partition, for best performance ensure no partitions are empty
23/06/08 06:19:21 WARN BulkPartitionTask: LightGBM task encountered empty partition, for best performance ensure no partitions are empty
23/06/08 06:19:21 WARN BulkPartitionTask:

space:                                                                           
{'baggingFraction': 0.6718426906712731, 'featureFraction': 0.7800717422292953, 'learningRate': 0.05403572759731447, 'maxDepth': 15.0, 'numLeaves': 68.0}
roc_auc:                                                                         
0.8293954363569715                                                               
 14%|█▍        | 2/14 [02:14<12:37, 63.11s/trial, best loss: -0.8307837932655073]

...

space:                                                                          
{'baggingFraction': 0.8313794850378632, 'featureFraction': 0.9324849413643902, 'learningRate': 0.02752462768428985, 'maxDepth': 10.0, 'numLeaves': 52.0}
roc_auc:                                                                        
0.8318145281831713                                                              
 86%|████████▌ | 12/14 [08:52<01:18, 39.23s/trial, best loss: -0.83410713739429]

23/06/08 06:26:42 WARN BulkPartitionTask: LightGBM task encountered empty partition, for best performance ensure no partitions are empty
23/06/08 06:26:42 WARN BulkPartitionTask: LightGBM task encountered empty partition, for best performance ensure no partitions are empty
23/06/08 06:26:42 WARN BulkPartitionTask: LightGBM task encountered empty partition, for best performance ensure no partitions are empty
23/06/08 06:26:42 WARN BulkPartitionTask: LightGBM task encountered empty partition, for best performance ensure no partitions are empty
23/06/08 06:26:42 WARN BulkPartitionTask: LightGBM task encountered empty partition, for best performance ensure no partitions are empty
23/06/08 06:26:42 WARN BulkPartitionTask: LightGBM task encountered empty partition, for best performance ensure no partitions are empty
23/06/08 06:26:42 WARN BulkPartitionTask: LightGBM task encountered empty partition, for best performance ensure no partitions are empty
23/06/08 06:26:42 WARN BulkPartitionTask:

space:                                                                          
{'baggingFraction': 0.6008509984746714, 'featureFraction': 0.6162818728770981, 'learningRate': 0.059826495368925, 'maxDepth': 15.0, 'numLeaves': 64.0}
roc_auc:                                                                        
0.8316581691936652                                                              
 93%|█████████▎| 13/14 [09:34<00:39, 39.98s/trial, best loss: -0.83410713739429]

23/06/08 06:27:26 WARN BulkPartitionTask: LightGBM task encountered empty partition, for best performance ensure no partitions are empty
23/06/08 06:27:26 WARN BulkPartitionTask: LightGBM task encountered empty partition, for best performance ensure no partitions are empty
23/06/08 06:27:26 WARN BulkPartitionTask: LightGBM task encountered empty partition, for best performance ensure no partitions are empty
23/06/08 06:27:26 WARN BulkPartitionTask: LightGBM task encountered empty partition, for best performance ensure no partitions are empty
23/06/08 06:27:26 WARN BulkPartitionTask: LightGBM task encountered empty partition, for best performance ensure no partitions are empty
23/06/08 06:27:26 WARN BulkPartitionTask: LightGBM task encountered empty partition, for best performance ensure no partitions are empty
23/06/08 06:27:26 WARN BulkPartitionTask: LightGBM task encountered empty partition, for best performance ensure no partitions are empty
23/06/08 06:27:26 WARN BulkPartitionTask:

space:                                                                          
{'baggingFraction': 0.9859360115622483, 'featureFraction': 0.9066672526263845, 'learningRate': 0.09066173097135129, 'maxDepth': 18.0, 'numLeaves': 40.0}
roc_auc:                                                                        
0.8321635629526517                                                              
100%|██████████| 14/14 [10:15<00:00, 43.95s/trial, best loss: -0.83410713739429]
{'baggingFraction': 0.878691825606384, 'featureFraction': 0.7775033179090377, 'learningRate': 0.049720884556106223, 'maxDepth': 14.0, 'numLeaves': 32.0}
[{'loss': -0.8307837932655073, 'status': 'ok'}, {'loss': -0.8293954363569715, 'status': 'ok'}, {'loss': -0.8326294197131474, 'status': 'ok'}, {'loss': -0.8320698191191266, 'status': 'ok'}, {'loss': -0.8324792327890168, 'status': 'ok'}, {'loss': -0.8315500238742644, 'status': 'ok'}, {'loss': -0.8303742006951714, 'status': 'ok'}, {'loss': -0.830753737990637, 'status': '

In [14]:
best_lgbm_classifier = LightGBMClassifier(
        numIterations=50, maxDepth=(int(lgbm_best['maxDepth'])),
        numLeaves=lgbm_best['numLeaves'],
        baggingFraction=lgbm_best['baggingFraction'],
        featureFraction=lgbm_best['featureFraction'],
        learningRate=lgbm_best['learningRate'],
        labelCol='TARGET'
)
best_lgbm_model = best_lgbm_classifier.fit(train_sdf_vectorized)
lgbm_predictions = best_lgbm_model.transform(val_sdf_vectorized)
roc_eval = BinaryClassificationEvaluator(labelCol='TARGET')
roc_auc = roc_eval.evaluate(lgbm_predictions)
print('roc_auc:', roc_auc)

23/06/08 06:28:06 WARN BulkPartitionTask: LightGBM task encountered empty partition, for best performance ensure no partitions are empty
23/06/08 06:28:06 WARN BulkPartitionTask: LightGBM task encountered empty partition, for best performance ensure no partitions are empty
23/06/08 06:28:06 WARN BulkPartitionTask: LightGBM task encountered empty partition, for best performance ensure no partitions are empty
23/06/08 06:28:06 WARN BulkPartitionTask: LightGBM task encountered empty partition, for best performance ensure no partitions are empty
23/06/08 06:28:06 WARN BulkPartitionTask: LightGBM task encountered empty partition, for best performance ensure no partitions are empty
23/06/08 06:28:06 WARN BulkPartitionTask: LightGBM task encountered empty partition, for best performance ensure no partitions are empty
23/06/08 06:28:06 WARN BulkPartitionTask: LightGBM task encountered empty partition, for best performance ensure no partitions are empty
23/06/08 06:28:06 WARN BulkPartitionTask:

roc_auc: 0.8340980134715614
